In [526]:
import open3d as o3d
import numpy as np
import multiprocessing as mp
from multiprocessing import Pool
import copy as cp
import open3d.core as o3c
import matplotlib.pyplot as plt
import pyransac3d as pyrsc
import time
import functions
from scipy.spatial.transform import Rotation
from iteration_utilities import deepflatten
from mpl_toolkits.mplot3d import Axes3D

In [527]:
#load pcd file, filter, downsample
#pcdn = o3d.io.read_point_cloud("loop.pcd")

voxel_size = 0.4
pcd = o3d.io.read_point_cloud("final_cropped_ground_align.pcd")
pcd.estimate_normals()
cl, ind = pcd.remove_statistical_outlier(nb_neighbors=20,
                                                    std_ratio=0.8)

pcd = pcd.select_by_index(ind)
pcd = pcd.voxel_down_sample(voxel_size=0.1)
pcd.estimate_normals()
pcd.orient_normals_consistent_tangent_plane(40)
#o3d.visualization.draw_geometries([pcd])

In [528]:
pcd_flat = functions.filter_by_normal(pcd)

#o3d.visualization.draw_geometries([pcd_flat])

In [529]:
oboxes = pcd_flat.detect_planar_patches(
normal_variance_threshold_deg=50,
coplanarity_deg=85,
outlier_ratio=0.75,
min_plane_edge_length=2,
min_num_points=10,
search_param=o3d.geometry.KDTreeSearchParamKNN(knn=50))

print("Detected {} patches".format(len(oboxes)))

geometries = []
meshes = []
for obox in oboxes:

    mesh = o3d.geometry.TriangleMesh.create_from_oriented_bounding_box(obox, scale=[1, 1, 0.0001])
    mesh.paint_uniform_color(obox.color)
    mesh.compute_triangle_normals()
    mesh.compute_vertex_normals()
    normals = np.asarray(mesh.vertex_normals)

    
    meshes.append(mesh)
    geometries.append(obox)



Detected 24 patches


In [530]:
hor_patches, ver_patches = devide_meshes_hor_ver(meshes)
o3d.visualization.draw_geometries(ver_patches)
o3d.visualization.draw_geometries(hor_patches)
print(len(hor_patches), len(ver_patches))

6 18


In [531]:


corr, corr_tuples = group_corresponding_meshes([],[],hor_patches, "horizontal")

corr, corr_tuples = group_corresponding_meshes(corr, corr_tuples, ver_patches, "vertical")


In [532]:




midpoints, marker_meshes = find_midpoints(corr_tuples)

print(midpoints)
o3d.visualization.draw_geometries(corr + marker_meshes)

[array([-1.11795710e+01,  5.40178564e+00,  5.84558413e-03]), array([-5.68762559,  5.1740348 ,  0.00577611]), array([-4.26459772e+01,  6.91534622e-01,  5.61329684e-03]), array([-3.93595241e+01,  3.83905978e-01,  5.37125959e-03]), array([-1.47350316e+01,  8.20477716e-02,  4.88058638e-03]), array([-2.29122973e+01,  2.67390782e-01,  5.20713948e-03]), array([-1.09587030e+01, -3.09172778e-03,  4.86511315e-03]), array([-3.39670614e+01, -2.81349493e+00,  1.83197874e-04]), array([-3.50376591e+01, -3.07940197e+00,  2.00021513e-04])]


In [533]:
#bb = pcd_flat.get_axis_aligned_bounding_box()
#bb.color = np.asarray([0,0,1])
#uniform_pc = create_uniform_pc_from_bb(bb, 0.2, [1,0,0])
hull, _ = pcd_flat.compute_convex_hull()

uniform_pc = hull_to_uniform_pc(hull, 0.2, [1,0,0])

o3d.visualization.draw_geometries([pcd_flat, uniform_pc, bb])

pcd_flat = create_uniform_pc(pcd_flat, 0.1, [0,0,0])
o3d.visualization.draw_geometries([pcd_flat])

In [534]:
# search multiple iterations until boundary found
search_radius = np.ceil(np.sqrt((voxel_size*2)**2+(voxel_size*2)**2)*10000)/10000

uniform_pc.paint_uniform_color([1,0,0])
pcd_flat.paint_uniform_color([0,0,0])

PointCloud with 4068 points.

In [535]:
def grow_void(pcd_grid, pcd_target, initial_seed, initial_set=set(), search_radius=0.2, stop_threshold=2, color=[0,0,1]):


    tree_grid = o3d.geometry.KDTreeFlann(pcd_grid)
    tree_target = o3d.geometry.KDTreeFlann(pcd_target)


    #initial search
    index = find_closest_vector_index(initial_seed, pcd_grid)
    center = pcd_grid.points[index]
    k_grid, k_target, idxu, idxf = knn_search_pointclouds(tree_grid, tree_target, center, search_radius)
    np.asarray(pcd_grid.colors)[idxu[1:], :] = color
    
    
    old_set = set()
    
    new_set = set(idxu[1:])
    different_elements = new_set-old_set-initial_set
    old_set = set(new_set) 
    
    while(different_elements):
    
                     
        
        
        for index in different_elements:
            new_center = pcd_grid.points[index]
            k_grid, k_target, idxu, idxf = knn_search_pointclouds(tree_grid, tree_target, new_center, search_radius)
            
            coords_grid = np.asarray(pcd_grid.points)[idxu]
            coords_target = np.asarray(pcd_target.points)[idxf]

            

            idx_up_grid, idx_up_target = get_ids_in_direction(idxu, idxf, coords_grid, coords_target, new_center, "up")
            idx_down_grid, idx_down_target = get_ids_in_direction(idxu, idxf, coords_grid, coords_target, new_center, "down")
            idx_left_grid, idx_left_target = get_ids_in_direction(idxu, idxf, coords_grid, coords_target, new_center, "left")
            idx_right_grid, idx_right_target = get_ids_in_direction(idxu, idxf, coords_grid, coords_target, new_center, "right")
            
            if (len(idx_up_target) < stop_threshold):
                new_set.update(idx_up_grid[1:])
                np.asarray(pcd_grid.colors)[idx_up_grid[1:], :] = color
            
            if (len(idx_down_target) < stop_threshold):
                new_set.update(idx_down_grid[1:])
                np.asarray(pcd_grid.colors)[idx_down_grid[1:], :] = color
            
            if (len(idx_left_target) < stop_threshold):
                new_set.update(idx_left_grid[1:])
                np.asarray(pcd_grid.colors)[idx_left_grid[1:], :] = color
            
            if (len(idx_right_target) < stop_threshold):
                new_set.update(idx_right_grid[1:])
                np.asarray(pcd_grid.colors)[idx_right_grid[1:], :] = color
        
        different_elements = new_set-old_set-initial_set
        old_set = set(new_set) 
        
    new_set = new_set.union(initial_set)
    
    return pcd_grid, new_set
        



In [536]:
def extract_void_area(pcd_grid, pcd_target, midpoints, known_points = set(),  search_radius=1, color=[0,0,1]):
    
    for points in midpoints:
        color = [0,0,1]
        void_area, known_points  = grow_void(pcd_grid, pcd_target, points, known_points, search_radius, color=color)
        #o3d.visualization.draw_geometries([void_area, pcd_flat]+marker_meshes)
    return void_area, known_points

void_area, known_points = extract_void_area(uniform_pc, pcd_flat, midpoints)
o3d.visualization.draw_geometries([void_area, pcd_flat]+marker_meshes)

In [537]:
valid_area = uniform_pc.select_by_index(list(known_points))
valid_area_voxel = o3d.geometry.VoxelGrid.create_from_point_cloud(valid_area, 0.2)
o3d.visualization.draw_geometries([valid_area_voxel, pcd_flat])